In [16]:
import pandas
import json
from pandas.io.json import json_normalize
from datetime import datetime
pandas.set_option('display.max_columns',None)
pandas.set_option('display.max_rows',None)
import requests
import itertools
from bs4 import BeautifulSoup
import numpy

portfolio_list = ['ACB','AMD','CCL','COST','CX','DIS','FANG',
              'GE','GRUB','MTCH','NLY','OKE',
              'PLD','PYPL','RBLX','SQ','TDOC','TSLA',
              'WWE','DNMR','BMBL','SHOP','WMT','TGT','EBAY',
              'CRSP','NKLA','LYB','RIO','IP','COP']

all_tickers = pandas.read_csv('stock_ticker_info.csv')

headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

# List Refinement #

focus_list = pandas.merge(pandas.DataFrame(portfolio_list,columns=['Ticker']),all_tickers,on='Ticker')[['Ticker','Name','Sector','Industry']]
sectors = focus_list['Sector'].unique()
industries = focus_list['Industry'].unique()
ticker_list = all_tickers[all_tickers['Industry'].isin(industries)]#['Ticker'].to_list()
ticker_list['key'] = ticker_list.Market_Cap.str.replace(r"[0-9/./()]",'')
ticker_list['value'] = pandas.to_numeric(ticker_list.Market_Cap.str.replace(r"[^0-9/^./^()]",''))
ticker_list['New_Market_Cap'] = numpy.where(ticker_list['key']=='-',0,
                                   numpy.where(ticker_list['key']=='B',
                                   ticker_list['value']*1000000000,
                                   ticker_list['value']*1000000))
# greater than 10 Billion Market Cap or original list
ticker_df = ticker_list
ticker_list = ticker_list[(ticker_list['New_Market_Cap']>10000000000) | (ticker_list['Ticker'].isin(portfolio_list))]['Ticker'].to_list()
rejoin_df = pandas.DataFrame(ticker_df)

C:\Users\dbetc\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\dbetc\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\dbetc\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

# Scraper to get all stock tickers in US markets
### Only need to run when refreshing the list (only to capture new listings, ipos, directs, etc...)

In [ ]:
# finviz_df = pandas.DataFrame()
# mylist = range(1,8000)

# for count in mylist[::20]:
#     url = "https://finviz.com/screener.ashx?v=111&r="+str(count)
#     finviz = BeautifulSoup(requests.get(url,headers=headers,allow_redirects=False).text,"lxml")
#     fin_table = pandas.read_html(str(finviz.find_all("table")[13]))
#     fin_table = fin_table[3].dropna(axis=0, thresh=4)
#     finviz_df = pandas.concat([finviz_df,fin_table],axis=0,sort=True)


# finviz_df.columns = ['Row','Ticker','Name','Sector','Industry','Country','Market_Cap','Current_P/E','Current_Price','Day_Chg','Current_Vol']
# finviz_df = finviz_df.drop(finviz_df[finviz_df.Ticker == 'Ticker'].index)
# finviz_df = finviz_df.drop_duplicates()
# finviz_df.to_csv('stock_ticker_info.csv')

# Scraper for Stock Financials

In [17]:
table2_df = pandas.DataFrame()
table1_df = pandas.DataFrame()

for symbol in ticker_list:
    url = "https://www.marketwatch.com/investing/stock/"+symbol+"/financials/balance-sheet"
    marketwatch = BeautifulSoup(requests.get(url,headers=headers,allow_redirects=False).text,"lxml")
    try:
        table2 = pandas.read_html(str(marketwatch.find_all("table")[5]))
        table2 = table2[0].dropna(axis=0, thresh=4)
        table2['ticker'] = symbol
        table2['PeriodType'] = 'Annual'
        table2_df = pandas.concat([table2_df,table2],axis=0,sort=True)
    except Exception:
        pass
    try:
        table1 = pandas.read_html(str(marketwatch.find_all("table")[4]))
        table1 = table1[0].dropna(axis=0, thresh=4)
        table1['ticker'] = symbol
        table1['PeriodType'] = 'Annual'
        table1_df = pandas.concat([table1_df,table1],axis=0,sort=True)
    except Exception:
        pass
    url = "https://www.marketwatch.com/investing/stock/"+symbol+"/financials/balance-sheet/quarter"
    marketwatch = BeautifulSoup(requests.get(url,headers=headers,allow_redirects=False).text,"lxml")
    try:
        table2 = pandas.read_html(str(marketwatch.find_all("table")[5]))
        table2 = table2[0].dropna(axis=0, thresh=4)
        table2['ticker'] = symbol
        table2['PeriodType'] = 'Quarterly'
        table2_df = pandas.concat([table2_df,table2],axis=0,sort=True)
    except Exception:
        pass
    try:
        table1 = pandas.read_html(str(marketwatch.find_all("table")[4]))
        table1 = table1[0].dropna(axis=0, thresh=4)
        table1['ticker'] = symbol
        table1['PeriodType'] = 'Quarterly'
        table1_df = pandas.concat([table1_df,table1],axis=0,sort=True)
    except Exception:
        pass
    
balance_sheets = pandas.concat([table2_df,table1_df],sort=True)
balance_sheets = pandas.melt(balance_sheets, id_vars=['Item Item','ticker','PeriodType'])
balance_sheets.columns = ['LineItem','Ticker','PeriodType','Period','Value']
balance_sheets = balance_sheets.dropna()
balance_sheets['Length'] = balance_sheets['Value'].str.len()
balance_sheets = balance_sheets.assign(Multiplier=balance_sheets.apply(lambda x: x['Value'][x['Length']-1:], axis=1))
balance_sheets['Neg'] = numpy.where(balance_sheets['Multiplier'] == ')','yes','no')
balance_sheets['key'] = balance_sheets.Value.str.replace(r"[0-9/./()]",'')
balance_sheets['Stripped_Value'] = balance_sheets.Value.str.replace(r"[^\d.]",'')
balance_sheets['Stripped_Value'] = pandas.to_numeric(balance_sheets['Stripped_Value'])
balance_sheets['Real_Value'] = numpy.where(balance_sheets.key=='B',balance_sheets['Stripped_Value']*1000000000,
                                           numpy.where(balance_sheets.key=='M',balance_sheets['Stripped_Value']*1000000,
                                                 numpy.where(balance_sheets.key=='K',balance_sheets['Stripped_Value']*1000,
                                                       numpy.where(balance_sheets.key=='%',balance_sheets['Stripped_Value']/100,
                                                             numpy.where(balance_sheets.key=='-%',balance_sheets['Stripped_Value']/-100,
                                                                   numpy.where(balance_sheets.key=='',balance_sheets['Stripped_Value'],
                                                                               numpy.where(balance_sheets.key=='-',balance_sheets['Stripped_Value'],
                                                                                           numpy.where(balance_sheets.key=='T',balance_sheets['Stripped_Value']*1000000000000,0))))))))
balance_sheets['Real_Value'] = numpy.where(balance_sheets.Neg=='yes',(balance_sheets['Real_Value']*-1)/1000000,balance_sheets['Real_Value']/1000000)
balance_sheets = balance_sheets[(['LineItem','Ticker','PeriodType','Period','Real_Value'])]
balance_sheets = balance_sheets.pivot_table(index=(['Period','Ticker','PeriodType']), columns='LineItem', values='Real_Value').reset_index()

asset_df = pandas.DataFrame()

for symbol in ticker_list:
    url = "https://www.marketwatch.com/investing/stock/"+symbol+"/financials"
    marketwatch = BeautifulSoup(requests.get(url,headers=headers,allow_redirects=False).text,"lxml")
    try:
        assets = pandas.read_html(str(marketwatch.find_all("table")[4]))
        assets = assets[0].dropna(axis=0, thresh=4)
        assets['ticker'] = symbol
        assets['PeriodType'] = 'Annual'
        asset_df = pandas.concat([asset_df,assets],axis=0,sort=True)
    except Exception:
        pass
    url = "https://www.marketwatch.com/investing/stock/"+symbol+"/financials/income/quarter"
    marketwatch = BeautifulSoup(requests.get(url).text,"lxml")
    try:
        assets = pandas.read_html(str(marketwatch.find_all("table")[4]))
        assets = assets[0].dropna(axis=0, thresh=4)
        assets['ticker'] = symbol
        assets['PeriodType'] = 'Quarterly'
        asset_df = pandas.concat([asset_df,assets],axis=0,sort=True)
    except Exception:
        pass
    
income = asset_df
income = pandas.melt(income, id_vars=['Item Item','ticker','PeriodType'])
income.columns = ['LineItem','Ticker','PeriodType','Period','Value']
income = income.dropna()
income['Length'] = income['Value'].str.len()
income = income.assign(Multiplier=income.apply(lambda x: x['Value'][x['Length']-1:], axis=1))
income['Neg'] = numpy.where(income['Multiplier'] == ')','yes','no')
income['key'] = income.Value.str.replace(r"[0-9/./()]",'')
income['Stripped_Value'] = income.Value.str.replace(r"[^\d.]",'')
income['Stripped_Value'] = pandas.to_numeric(income['Stripped_Value'])
income['Real_Value'] = numpy.where(income.key=='B',income['Stripped_Value']*1000000000,
                                           numpy.where(income.key=='M',income['Stripped_Value']*1000000,
                                                 numpy.where(income.key=='K',income['Stripped_Value']*1000,
                                                       numpy.where(income.key=='%',income['Stripped_Value']/100,
                                                             numpy.where(income.key=='-%',income['Stripped_Value']/-100,
                                                                   numpy.where(income.key=='',income['Stripped_Value'],
                                                                               numpy.where(income.key=='-',income['Stripped_Value'],
                                                                                           numpy.where(income.key=='T',income['Stripped_Value']*1000000000000,0))))))))
income['Real_Value'] = numpy.where(income.Neg=='yes',(income['Real_Value']*-1)/1000000,income['Real_Value']/1000000)
income = income[(['LineItem','Ticker','PeriodType','Period','Real_Value'])]
income = income.pivot_table(index=(['Period','Ticker','PeriodType']), columns='LineItem', values='Real_Value').reset_index()

table2_df = pandas.DataFrame()
table1_df = pandas.DataFrame()
table3_df = pandas.DataFrame()

for symbol in ticker_list:
    url = "https://www.marketwatch.com/investing/stock/"+symbol+"/financials/cash-flow"
    marketwatch = BeautifulSoup(requests.get(url,headers=headers,allow_redirects=False).text,"lxml")
    try:
        table2 = pandas.read_html(str(marketwatch.find_all("table")[5]))
        table2 = table2[0].dropna(axis=0, thresh=4)
        table2['ticker'] = symbol
        table2['PeriodType'] = 'Annual'
        table2_df = pandas.concat([table2_df,table2],axis=0,sort=True)
    except Exception:
        pass
    try:
        table1 = pandas.read_html(str(marketwatch.find_all("table")[4]))
        table1 = table1[0].dropna(axis=0, thresh=4)
        table1['ticker'] = symbol
        table1['PeriodType'] = 'Annual'
        table1_df = pandas.concat([table1_df,table1],axis=0,sort=True)
    except Exception:
        pass
    try:
        table3 = pandas.read_html(str(marketwatch.find_all("table")[6]))
        table3 = table3[0].dropna(axis=0, thresh=4)
        table3['ticker'] = symbol
        table3['PeriodType'] = 'Annual'
        table3_df = pandas.concat([table3_df,table3],axis=0,sort=True)
    except Exception:
        pass
    url = "https://www.marketwatch.com/investing/stock/"+symbol+"/financials/cash-flow/quarter"
    marketwatch = BeautifulSoup(requests.get(url,headers=headers,allow_redirects=False).text,"lxml")
    try:
        table2 = pandas.read_html(str(marketwatch.find_all("table")[5]))
        table2 = table2[0].dropna(axis=0, thresh=4)
        table2['ticker'] = symbol
        table2['PeriodType'] = 'Quarterly'
        table2_df = pandas.concat([table2_df,table2],axis=0,sort=True)
    except Exception:
        pass
    try:
        table1 = pandas.read_html(str(marketwatch.find_all("table")[4]))
        table1 = table1[0].dropna(axis=0, thresh=4)
        table1['ticker'] = symbol
        table1['PeriodType'] = 'Quarterly'
        table1_df = pandas.concat([table1_df,table1],axis=0,sort=True)
    except Exception:
        pass
    try:
        table3 = pandas.read_html(str(marketwatch.find_all("table")[6]))
        table3 = table3[0].dropna(axis=0, thresh=4)
        table3['ticker'] = symbol
        table3['PeriodType'] = 'Quarterly'
        table3_df = pandas.concat([table3_df,table3],axis=0,sort=True)
    except Exception:
        pass
    
cash_flow = pandas.concat([pandas.concat([table2_df,table1_df],sort=True),table3_df],sort=True)
cash_flow = pandas.melt(cash_flow, id_vars=['Item Item','ticker','PeriodType'])
cash_flow.columns = ['LineItem','Ticker','PeriodType','Period','Value']
cash_flow = cash_flow.dropna()
cash_flow['Length'] = cash_flow['Value'].str.len()
cash_flow = cash_flow.assign(Multiplier=cash_flow.apply(lambda x: x['Value'][x['Length']-1:], axis=1))
cash_flow['Neg'] = numpy.where(cash_flow['Multiplier'] == ')','yes','no')
cash_flow['key'] = cash_flow.Value.str.replace(r"[0-9/./()]",'')
cash_flow['Stripped_Value'] = cash_flow.Value.str.replace(r"[^\d.]",'')
cash_flow['Stripped_Value'] = pandas.to_numeric(cash_flow['Stripped_Value'])
cash_flow['Real_Value'] = numpy.where(cash_flow.key=='B',cash_flow['Stripped_Value']*1000000000,
                                           numpy.where(cash_flow.key=='M',cash_flow['Stripped_Value']*1000000,
                                                 numpy.where(cash_flow.key=='K',cash_flow['Stripped_Value']*1000,
                                                       numpy.where(cash_flow.key=='%',cash_flow['Stripped_Value']/100,
                                                             numpy.where(cash_flow.key=='-%',cash_flow['Stripped_Value']/-100,
                                                                   numpy.where(cash_flow.key=='',cash_flow['Stripped_Value'],
                                                                               numpy.where(cash_flow.key=='-',cash_flow['Stripped_Value'],
                                                                                           numpy.where(cash_flow.key=='T',cash_flow['Stripped_Value']*1000000000000,0))))))))
cash_flow['Real_Value'] = numpy.where(cash_flow.Neg=='yes',(cash_flow['Real_Value']*-1)/1000000,cash_flow['Real_Value']/1000000)
cash_flow = cash_flow[(['LineItem','Ticker','PeriodType','Period','Real_Value'])]
cash_flow = cash_flow.pivot_table(index=(['Period','Ticker','PeriodType']), columns='LineItem', values='Real_Value').reset_index()

final = pandas.merge(pandas.merge(balance_sheets,income,on=['Period','Ticker']),cash_flow,on=['Period','Ticker'])
final['Period'] = pandas.to_datetime(final['Period'])
final = final.fillna(0)
final['Update'] = pandas.Timestamp.now()
final = pandas.merge(final,ticker_df,how='left',on='Ticker')

# final.to_csv('stock_financials.csv')

# Scraper for Analyst Opinion

In [18]:
table2_df = pandas.DataFrame()
table1_df = pandas.DataFrame()

for symbol in ticker_list:
    url = "https://www.marketwatch.com/investing/stock/"+symbol+"/analystestimates?mod=mw_quote_tab"
    marketwatch = BeautifulSoup(requests.get(url,headers=headers,allow_redirects=False).text,"lxml")
    try:
        table2 = pandas.read_html(str(marketwatch.find_all("table")[7]))
        table2 = table2[0].dropna(axis=0, thresh=4)
        table2['ticker'] = symbol
        table2_df = pandas.concat([table2_df,table2],axis=0,sort=True)
    except Exception:
        pass
    try:
        table1 = pandas.read_html(str(marketwatch.find_all("table")[9]))
        table1 = table1[0].dropna(axis=0, thresh=4)
        table1['ticker'] = symbol
        table1_df = pandas.concat([table1_df,table1],axis=0,sort=True)
    except Exception:
        pass
    
analysts = pandas.concat([table2_df,table1_df],sort=True)
analysts = pandas.melt(analysts, id_vars=['Unnamed: 0','ticker'])
analysts.columns = ['Category','Ticker','Period','Value']
analysts = analysts.dropna()
analysts = analysts[(['Category','Ticker','Period','Value'])]
analysts = analysts.set_index('Category')
analysts = analysts.pivot_table(index=(['Period','Ticker']), columns='Category', values='Value',aggfunc='first').reset_index()

analysts.to_csv('stock_analysts.csv')

# Old scraper template

In [64]:
# empty = []

# for symbol in ticker_list:
#     try:
#         url = "https://www.marketwatch.com/investing/stock/"+symbol+"/company-profile?mod=mw_quote_tab"
#         marketwatch = BeautifulSoup(requests.get(url,headers=headers,allow_redirects=False).text,"lxml")
#         ind = marketwatch.find_all('span',{'primary'})[6].text
#         sec = marketwatch.find_all('span',{'primary'})[7].text
#         empty.append([symbol,ind,sec])
#     except Exception:
#         pass
# stock_info = pandas.DataFrame(empty,columns=['Ticker','Industry','Sector'])